<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D8%A8%D9%86%D8%A7%D8%A1_%D8%A7%D9%84%D8%B3%D8%AA%D8%B1%D8%A7%D8%AA%D9%8A%D8%AC%D9%8A%D8%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # حساب مناطق الطلب
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

        # الحصول على مناطق الطلبات
        مناطق_طلبات_الشراء = data['مناطق_طلبات_الشراء'].dropna().tolist()  # تأكد من وجود البيانات
        مناطق_طلبات_البيع = data['مناطق_طلبات_البيع'].dropna().tolist()  # تأكد من وجود البيانات

        # جلب الأخبار ذات التأثير العالي
        news_times = fetch_high_impact_news()

        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)  # حساب مؤشر القوة النسبية
        data = calculate_moving_average(data)  # حساب المتوسط المتحرك
        data = calculate_moving_average20(data)
        premium, discount = calculate_premium_discount(data)  # حساب العلاوة والخصم
        data['Premium'] = premium
        data['Discount'] = discount

        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)
        data = calculate_stochastic(data)  # حساب مؤشر Stochastic

        # جلب آخر صف من البيانات
        last_row = data.iloc[-1]

        # حساب عدد الصفقات المفتوحة
        open_positions_count = len(mt5.positions_get(symbol="XAUUSD"))

        # الشروط لفتح أوامر البيع
        if open_positions_count < 5:
            if (last_row['target'] < last_row['close'] and
                last_row['Lower_Band'] < last_row['close'] < last_row['Middle_Band'] and
                last_row['Signal_Line'] > last_row['MACD'] and
                last_row['close'] < مناطق_طلبات_الشراء[-1][1] and
                last_row['Premium'] < last_row['Discount'] and
                50 > last_row['RSI'] > 30 and
                last_row['%K'] < last_row['%D'] and
                last_row['MA'] > last_row['MA20'] > last_row['close']):
                place_order("XAUUSD", "sell", volume=0.01)
                print("تم فتح أمر بيع.")

            elif (last_row['target'] > last_row['close'] and
                  last_row['Middle_Band'] < last_row['close'] < last_row['Upper_Band'] and
                  last_row['Signal_Line'] < last_row['MACD'] and
                  مناطق_طلبات_الشراء[-1][1] > last_row['close'] > مناطق_طلبات_البيع[-1][1] and
                  last_row['Premium'] < last_row['Discount'] and
                  50 > last_row['RSI'] > 30 and
                  last_row['%K'] > last_row['%D'] and
                  last_row['MA20'] < last_row['close'] < last_row['MA']):
                place_order("XAUUSD", "sell", volume=0.01)
                print("تم فتح أمر بيع.")

        # الشروط لفتح أوامر الشراء
        if open_positions_count < 5:
            if (last_row['target'] > last_row['close'] and
                last_row['Middle_Band'] < last_row['close'] < last_row['Upper_Band'] and
                last_row['Signal_Line'] < last_row['MACD'] and
                last_row['close'] > مناطق_طلبات_البيع[-1][1] and
                last_row['Premium'] > last_row['Discount'] and
                50 > last_row['RSI'] < 70 and
                last_row['%K'] > last_row['%D'] and
                last_row['MA'] < last_row['MA20'] < last_row['close']):
                place_order("XAUUSD", "buy", volume=0.01)
                print("تم فتح أمر شراء.")

            elif (last_row['target'] < last_row['close'] and
                  last_row['Lower_Band'] < last_row['close'] < last_row['Middle_Band'] and
                  last_row['Signal_Line'] > last_row['MACD'] and
                  مناطق_طلبات_الشراء[-1][1] > last_row['close'] < مناطق_طلبات_بيع[-1][1] and
                  last_row['Premium'] > last_row['Discount'] and
                  50 > last_row['RSI'] < 70 and
                  last_row['%K'] < last_row['%D'] and
                  last_row['MA'] < last_row['close'] < last_row['MA20']):
                place_order("XAUUSD", "buy", volume=0.01)
                print("تم فتح أمر شراء.")

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")
